# Streamlined framework of EDA for classification models

This project was designed to streamline the explorarory data analysis of common datasets that contained categorical, numeric and time-based data. The steps below focus on the common cleaning and feature engineering done on datasets to prep them for ML models. 

Since 70% of time taken in data science projects focus on EDA, this tries to recuce time by passing over the commmon steps taken. This was specifically not attuned to object-oriented programming as each cell has its own specific feature that needs to be monitored before continuing with the next step.     

### Concepts taken from: 

The Essential Toolbox for Data Cleaning  - https://towardsdatascience.com/the-simple-yet-practical-data-cleaning-codes-ad27c4ce0a38

Code snippets from CHRIS ALBON-
https://chrisalbon.com/


Data Cleaning with Python - https://towardsdatascience.com/data-cleaning-in-python-the-ultimate-guide-2020-c63b88bf0a0d

EDA for Feature Selection in ML - http://services.google.com/fh/files/misc/exploratory_data_analysis_for_feature_selection_in_machine_learning.pdf

Pythonic Data Cleaning With Pandas and NumPy - https://realpython.com/python-data-cleaning-numpy-pandas/

Tour of Evaluation Metrics for Imbalanced Classification - https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/

A Starter Pack to EDA - https://towardsdatascience.com/a-starter-pack-to-exploratory-data-analysis-with-python-pandas-seaborn-and-scikit-learn-a77889485baf

## NOTE 
Some of the topics have been color coded so as to emphasize its use in the code

<span style="color:red">**RED**</span> - Warnings before using <p><p> 
<span style="color:blue">**BLUE**</span> - An option to choose from <p><p>
<span style="color:green">**GREEN**</span> - Experimental development - would like some insights <p><p>


## <span style="color:red">**IMPORTANT**</span>

Features  which require manual addtion of specific column variables are referenced as **col1** 

### Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns 

import matplotlib.pyplot as plt
import scipy.stats as ss

from datetime import date, time
from dateutil.parser import parse

from math import log
from math import ceil

from sklearn.feature_selection import chi2
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items',None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [ ]:
#Color palette for Box plot
c_palette = ['tab:blue', 'tab:orange']
c_palette3 = ['tab:blue', 'tab:orange', 'tab:green']

### Read the data [CSV]

#### Pandas has a lot more features for reading csv data 

pd.read_csv(*filepath_or_buffer*, *sep=’*, ‘, delimiter=None, header=’infer’, names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, *nrows=None*, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression=’infer’, thousands=None, decimal=b’.’, lineterminator=None, quoting=0, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, skipfooter=0, doublequote=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)

In [ ]:
#File location
DATA_FOLDER = ''

In [ ]:
df= pd.read_csv(os.path.join(DATA_FOLDER,'bank-additional-full.csv'), sep=';')

### Combine seperate dataframes into one

#### Merging data is done depending on the primary key and what type of join

df.merge(self, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None) 

In [ ]:
df.merge(self, col1, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None) 

### Initial features

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# import plotly.figure_factory as ff
# figure = ff.create_scatterplotmatrix(df,diag='histogram')

In [ ]:
#sns.pairplot(df)


### Find and drop duplicated values


<span style="color:red">**Check if the duplicated values mean anything significant**</span>

In [ ]:
# generate count statistics of duplicate entries
if len(df[df.duplicated()]) > 0:
    print("No. of duplicated entries: ", len(df[df.duplicated()]))
    print(df[df.duplicated(keep=False)].sort_values(by=list(df.columns)).head())
else:
    print("No duplicated entries found")

In [ ]:
#Drop the duplicate
df.drop_duplicates(inplace=True)

### Null values

In [ ]:
df.isnull().sum().sort_values(ascending=False)

### In-depth analysis on missing values

In [ ]:
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

### Working on missing values

#### <span style="color:blue">**(1) Drop Missing Rows**</span> 

In [ ]:
# drop rows with a lot of missing values.
ind_missing = df[df['col1'] > 35].index
ind_missing

In [ ]:
df_less_missing_rows = df.drop(ind_missing, axis=0)

#### <span style="color:blue">**(2) Drop Missing Feature**</span> 

In [ ]:
# drop the feature entirely
cols_to_drop = ['col1']
df_less_hos_beds_raion = df.drop(cols_to_drop, axis=1)

#### <span style="color:green">**Imputation is done later when we have a better idea about the numeric data**</span> 

### Identify the target variable and shift it to the beginning

In [ ]:
target = df['col1']
target_name=target.name

In [ ]:
df.drop(labels=target_name, axis=1,inplace = True)
df.insert(0,target_name,target)
df.head()

### General Statistical Insights

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.dtypes.unique()

<span style="color:green">**Check for imbalances in the classifiers**</span>

Each method is applicable based on the dataset.
Taken from The Machine Learning Mastery website - Evaluation Metrics for Imbalanced Classification


<ul>
<li><strong>Are you predicting probabilities?</strong>
<ul>
<li><strong>Do you need class labels?</strong>
<ul>
<li><strong>Is the positive class more important?</strong>
<ul>
<li>Use Precision-Recall AUC</li>
</ul>
</li>
<li><strong>Are both classes important?</strong>
<ul>
<li>Use ROC AUC</li>
</ul>
</li>
</ul>
</li>
<li><strong>Do you need probabilities?</strong>
<ul>
<li>Use Brier Score and Brier Skill Score</li>
</ul>
</li>
</ul>
</li>
<li><strong>Are you predicting class labels?</strong>
<ul>
<li><strong>Is the positive class more important?</strong>
<ul>
<li><strong>Are False Negatives and False Positives Equally Important?</strong>
<ul>
<li>Use F1-Measure</li>
</ul>
</li>
<li><strong>Are False Negatives More Important?</strong>
<ul>
<li>Use F2-Measure</li>
</ul>
</li>
<li><strong>Are False Positives More Important?</strong>
<ul>
<li>Use F0.5-Measure</li>
</ul>
</li>
</ul>
</li>
<li><strong>Are both classes important?</strong>
<ul>
<li><strong>Do you have &lt; 80%-90% Examples for the Majority Class?&nbsp;</strong>
<ul>
<li>Use Accuracy</li>
</ul>
</li>
<li><strong>Do you have &gt; 80%-90% Examples for the Majority Class?&nbsp;</strong>
<ul>
<li>Use G-Mean</li>
</ul>
</li>
</ul>
</li>
</ul>
</li>
</ul>

### Column-wise analysis

In [ ]:
unique_values= df.nunique()
unique_values

Checks for columns with varying data values

In [ ]:
num_rows = len(df.index)
low_information_cols = [] #

for col in df.columns:
    cnts = df[col].value_counts(dropna=False)
    top_pct = (cnts/num_rows).iloc[0]
    
    if top_pct < 0.75 and top_pct > 0.20:
        low_information_cols.append(col)
        print('{0}: {1:.5f}%'.format(col, top_pct*100))
        print(cnts)
        print()

In [ ]:
#plot for unique analysis
for i in df.columns:
    plt.figure()
    plt.title(f'{i} - {unique_values[i]} unique values')
    values=pd.value_counts(df[i]).plot.bar()   

### Change the index of the dataframe as a unique column

In [ ]:
# Find columns with unique values
unique_columns =[index for  index,values in unique_values.items() if values==len(df) ]
print(f'{unique_columns if unique_columns !=[] else None}')

In [ ]:
df.set_index('col1',inplace=True)
df.head()

### Convert numeric to categorical feature

Sometimes categorical values are denoted as numeric (0 and 1). It is better to label them as categorical before splitting.

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
to_categorical=["col1"]
df[to_categorical]=df[to_categorical].astype(str)

### Overview of summary based on the target variable

In [ ]:
grouped_values = df.groupby(target)
grouped_summary=grouped_values.mean()
grouped_summary

### Rename columns if needed

In [ ]:
replace_columns={'col1':'col1'}
df.rename(columns=replace_columns,inplace=True)
df.head()

### Drop the target variable from the dataframe

Since we have a target stored as a variable, it is better to seperate it from the rest of the dataframe

In [ ]:
df.drop(columns=target.name,inplace=True)
df.head()

### Check if the target variable needs to be encoded

In [ ]:
le = LabelEncoder()
target = le.fit_transform(target)
target[:5]

### Get numeric and categorical features

NOTE: Categorical features will also contain timestamps as there may be a chance that it exists as a string datatype

In [ ]:
df_numeric=df.select_dtypes(include=['float64', 'int64'])
df_categorical=df.select_dtypes(exclude=['float64', 'int64'])

## Categorical Data

### Remove extra white space in text columns

In [ ]:
for col in df_categorical.columns:
    df_categorical[col] = df_categorical[col].str.strip()

### Bar plot of categorical features

In [ ]:
df_categorical.columns

In [ ]:
df_categorical.info()

In [ ]:
def categorical_summarized(dataframe, x=None, y=None, hue=None, palette='Set1', ax=None, order=None, verbose=True):
    '''
    Helper function that gives a quick summary of a given column of categorical data
    Arguments
    =========
    dataframe: pandas dataframe
    x: str. horizontal axis to plot the labels of categorical data, y would be the count
    y: str. vertical axis to plot the labels of categorical data, x would be the count
    hue: str. if you want to compare it another variable (usually the target variable)
    palette: array-like. Colour of the plot
    Returns
    =======
    Quick Stats of the data and also the count plot
    '''
    if x == None:
        column_interested = y
    else:
        column_interested = x
    series = dataframe[column_interested]
    print ('\t', column_interested)
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('='*80)
        print(series.value_counts())

    sns.countplot(x=x, y=y, hue=hue, data=dataframe, palette=palette)
    plt.show()
    print('\n'*2)

### Univariate analysis

In [ ]:
for col in df_categorical.columns:
    categorical_summarized(df_categorical,y=col)

### Bivariate analysis

In [ ]:
for col in df_categorical.columns:
    categorical_summarized(df_categorical,y=col,hue=target)

#### Impute missing categorical values with mode

In [ ]:
# impute the missing values and create the missing value indicator variables for each non-numeric column.
non_numeric_cols = df_categorical.columns.values

for col in non_numeric_cols:
    missing = df_categorical[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print('imputing missing values for: {}'.format(col))
        df_categorical['{}_ismissing'.format(col)] = missing
        
        top = df_categorical[col].describe()['top'] # impute with the most frequent value.
        df_categorical[col] = df_categorical[col].fillna(top)

## Chi square analysis

Tests for independence of categorical variables. Requires all categorical variables to be **label encoded** before analysis

In [ ]:
chi_square = df_categorical.apply(LabelEncoder().fit_transform)
chi_scores = chi2(chi_square,target)
chi_scores

In [ ]:
p_values = pd.Series(chi_scores[1],index = chi_square.columns)
p_values.sort_values(ascending = False , inplace = True)
p_values.plot.bar()

### Encoding categorical data

<span style="color:red">**Choose relevant features and encoding methods**</span>

In [ ]:
df_categorical.head()

#### <span style="color:blue">** (1) Label Encoder **</span> 

In [ ]:
label_encoded_variables=['col1']
le = LabelEncoder()
label_encoded=df_categorical[label_encoded_variables].apply(lambda x: le.fit_transform(x))
label_encoded.columns=[label+"_label_encoded" for label in label_encoded_variables]

# Convert to Integer values

In [ ]:
df_categorical.drop(labels=label_encoded_variables, axis=1,inplace=True)
df_categorical[label_encoded.columns] = label_encoded
df_categorical.head()

#### <span style="color:blue">** (2) Label Encoder with Weights  **</span> 

In [ ]:
num_encoded_variables=['col1']
num_encode = {'col1' : {'YES':1, 'NO':0},
              'col1'  : {'WON':1, 'LOSE':0, 'DRAW':0}}  

In [ ]:
label_encoded=df_categorical[num_encoded_variables].replace(num_encode) 
label_encoded.columns=[label+"_label_encoded" for label in num_encoded_variables] 
label_encoded.head()

In [ ]:
df_categorical.drop(labels=num_encoded_variables, axis=1,inplace=True)
df_categorical[label_encoded.columns] = label_encoded
df_categorical.head()

#### <span style="color:blue">** (3) One Hot Encoder  **</span> 

In [ ]:
one_hot_variables=['col1']
df_categorical=pd.get_dummies(data=df_categorical, columns=one_hot_variables,dummy_na=True)
df_categorical.head()

### Date-Time data 

<span style="color:red">**Make sure that the data related to time is a relevant feature (Classification models cannot use datetime variables)**</span>

#### Convert timestamp(from string to datetime format)

In [ ]:
df_categorical['timestamp']=pd.to_datetime(df_categorical['col1'], format='%Y-%m-%d %H:%M:%S.%f'))

#### Pull Day, month year if the timestamp is a string

In [ ]:
df['date'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d')
df_categorical['year'] = df_categorical['date'].dt.year
df_categorical['month'] = df_categorical['date'].dt.month
df_categorical['day'] = df_categorical['date'].dt.day
df['weekday'] = df['timestamp_dt'].dt.weekday

In [ ]:
df_categorical['hour'] = df_categorical['date'].dt.hour
df_categorical['minute'] = df_categorical['date'].dt.minute

#### Show Days Of The Week

In [ ]:
df_categorical['weekday'] = df_categorical.date.weekday()

#### Difference between two dates

In [ ]:
df_categorical['date_difference']=abs((d2 - d1).days)

#### Drop the orginal timestamp columns

In [ ]:
del(df_categorical['col1'])

## Finalized Categorical columns

In [ ]:
df_categorical.head()

## Numeric data

### Univariate analysis

### Histogram of numeric features

In [ ]:
df_numeric.head()

In [ ]:
df_numeric.hist(figsize=(20, 20), bins=10, xlabelsize=8, ylabelsize=8);

### General Statistical Insights

In [ ]:
def quantitative_summarized(dataframe, x=None, y=None, hue=None, palette='Set1', ax=None, order=None, verbose=True):
    '''
    Helper function that gives a quick summary of quantattive data
    Arguments
    =========
    dataframe: pandas dataframe
    x: str. horizontal axis to plot the labels of categorical data (usually the target variable)
    y: str. vertical axis to plot the quantitative data
    hue: str. if you want to compare it another categorical variable (usually the target variable if x is another variable)
    palette: array-like. Colour of the plot
    Returns
    =======
    Quick Stats of the data and also the box plot of the distribution
    '''
    series = dataframe[y]
    print ('\t', y)
    print(series.describe())
    print('mode: ', series.mode())
    print('Unique values: ', series.unique().size)
    if verbose:
        print('\n'*2)
        print(series.value_counts())
    
    sns.boxplot(x=x, y=y, hue=hue, data=dataframe,
                palette=palette, order=order, ax=ax)
                    
    plt.show()
    print('\n'+'#'*80+'\n')

### Univariate numeric analysis

In [ ]:
for col in df_numeric.columns:
    quantitative_summarized(df_numeric,y=col, verbose=False)

### Bivariate numeric analysis

In [ ]:
for col in df_numeric.columns:
    quantitative_summarized(dataframe= df_numeric, y = col, palette=c_palette, x = target, verbose=False)

### Multivariate numeric analysis

Use to see which 2 columns that sould be analyzed based on target column

In [ ]:
quantitative_summarized(dataframe= df_numeric, y = 'col1', x = target, hue = 'col1', palette=c_palette3, verbose=False)

### Binning numeric columns

Either by equal range/width (cut) or by equal frequencies(qcut)

In [ ]:
#If based on equal width, Use Sturge’s Rule to estimate the number of bins

sturge_value=1 + 3.322 *log(df_numeric.size)
print ("Optimal bins:", ceil(sturge_value))


In [ ]:
#Shows estimation
pd.cut(df_numeric['col1'],44).value_counts()

In [ ]:
#Model friendly version - Labeled within binned category
df_numeric_binned= pd.cut(df_numeric['col1'],44,labels=False)


#Side-by-side comparison
pd.DataFrame({'col1':df_numeric['col1'], 'col1_binned':df_numeric_binned}).head()

In [ ]:
df_numeric['col1'].plot(kind='hist')

In [ ]:
df_numeric_binned.plot(kind='col1')

In [ ]:
#append to the original dataframe

binned_name=df_numeric_binned.name
df_numeric.drop(labels=binned_name, axis=1,inplace = True)
df_numeric= pd.concat([df_numeric, df_numeric_binned], axis=1)
df_numeric.head()

### Handling outliers 

In [ ]:
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1
pd.DataFrame([Q1,Q3,IQR])

In [ ]:
# Shows all outliers
df_numeric_outliers = df_numeric[((df_numeric < (Q1 - 1.5 * IQR)) |(df_numeric > (Q3 + 1.5 * IQR))).any(axis=1)]
df_numeric_outliers.head()
print (df_numeric_outliers.head())
print ('\n',df_numeric_outliers.shape)


In [ ]:
#This is present to remove any of the outliers 
df_numeric = df_numeric[~df_numeric.index.isin(df_numeric_outliers.index)]
df_numeric.shape

### Dataframe after eliminating outliers

In [ ]:
df_numeric.info()

In [ ]:
df_numeric.describe()

### Skewedness and Kurtosis distributions

### Skewedness

* If skewness is less than −1 or greater than +1, the distribution is highly skewed.
* If skewness is between −1 and −½ or between +½ and +1, the distribution is moderately skewed.
* If skewness is between −½ and +½, the distribution is approximately symmetric.

### Kurtosis 

* A normal distribution has kurtosis exactly 3 (excess kurtosis exactly 0). Any distribution with kurtosis ≈3 (excess ≈0) is called mesokurtic.
* A distribution with kurtosis <3 (excess kurtosis <0) is called platykurtic. Compared to a normal distribution, its tails are shorter and thinner, and often its central peak is lower and broader.
* A distribution with kurtosis >3 (excess kurtosis >0) is called leptokurtic. Compared to a normal distribution, its tails are longer and fatter, and often its central peak is higher and sharper.

In [ ]:
df_numeric_orientation = pd.DataFrame({"Skewedness":df_numeric.skew(),"Kurtosis":df_numeric.kurt()})
df_numeric_orientation

### Reduce memory load by converting to 32-bits

<span style="color:red">**Confirm that the data values do not matter when downsizing**</span>

In [ ]:
# Change the dtypes (int64 -> int32)
df_numeric[['col1']] = df_numeric[['col1']].astype('int32')

# Change the dtypes (float64 -> float32)
df_numeric[['col1']] = df_numeric[['col1']].astype('float32')

### Impute all missing numeric data with Median 


In [ ]:
# impute the missing values and create the missing value indicator variables for each numeric column.
numeric_cols = df_numeric.columns.values

for col in numeric_cols:
    missing = df_numeric[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print(f'imputing missing values for: {col} - Missing {num_missing} values')

        med = df_numeric[col].median()
        df_numeric[col] = df_numeric[col].fillna(med)

print("All collumns checked") 

### Find qunatiles for each numeric value 

In [ ]:
quantile_check=df_numeric.quantile([.1, .25, .5, .75], axis = 0)
quantile_check

### Normalize Numeric features

#### MinMax Scaler

In [ ]:
cols=['col1']
scaled_variables = df_numeric[cols].values.astype(float)
min_max_scaler = MinMaxScaler()
min_max_scaled = min_max_scaler.fit_transform(scaled_variables)
min_max_scaled[:5]

In [ ]:
bc_cols = [f'min_max_{c}' for c in cols]
df_numeric=pd.concat([df_numeric, pd.DataFrame(min_max_scaled, columns=bc_cols)], axis='columns')
ds.head()

In [ ]:
#Original Visualization
df_numeric[cols].hist(layout=(2, 6), figsize=(15, 4));

In [ ]:
#Min_max transformed
df_numeric[bc_cols].hist(layout=(2, 6), figsize=(15, 4), color='orange', alpha=.5);

In [ ]:
#Choose to drop the original
df_numeric.drop(['col1'], axis=1, inplace=True) 

#### Yeo-Johnson transform


In [ ]:
cols=['col1']
scaled_variables = df_numeric[cols].values.astype(float)
yeo_johnson_scaler = PowerTransformer(method='yeo-johnson')
yeo_scaled = yeo_johnson_scaler.fit_transform(scaled_variables)
yeo_scaled[:5]

In [ ]:
bc_cols = [f'yeo_johnson_{c}' for c in cols]
df_numeric=pd.concat([df_numeric, pd.DataFrame(yeo_scaled, columns=bc_cols)], axis='columns')
ds.head()  

In [ ]:
#Original Visualization
df_numeric[cols].hist(layout=(2, 6), figsize=(15, 4));

In [ ]:
#Yeo-Johnsontransformed
df_numeric[bc_cols].hist(layout=(2, 6), figsize=(15, 4), color='orange', alpha=.5);

In [ ]:
#Choose to drop the original
df_numeric.drop(['col1'], axis=1, inplace=True) 

#### DataFrame transformations

<span style="color:red">**Make sure the relevant columns are transformed accordingly**</span>

In [ ]:
#Create a new colum based on the sum of two values
df_numeric['Total_col'] = df_numeric['col1'] + df_numeric['col2']

### Combine all the features

In [ ]:
df_target= pd.DataFrame(target.astype(int),columns=[target_name])
df_target.head()

In [ ]:
df_combined= pd.concat([df_numeric, df_categorical,df_target], axis=1)

In [ ]:
df_combined.head()

In [ ]:
#Checking if there are any null values in the final dataframe.
df_combined.info()

In [ ]:
# Removing any null values as they were removed due to a numeric/ categorical abnormality
df_combined.dropna(axis=0,inplace = True)

## Saving the overall dataframe

#### <span style="color:blue">**(1) As a CSV**</span> 

In [ ]:
df_combined.to_csv("transformed_data.csv",index=False)
print("Dataframe stored as a CSV")

#### <span style="color:blue">**(2) As a pickle**</span> 

In [ ]:
df_combined.to_pickle("./transformed_data.pkl")
print("Dataframe stored as a pickle")

## Conclusion

There are a lot of detailed feature analysis that needs to be performed on datasets before they can be deployed as a perfect dataset. This framework brings to light some of the more common EDA steps that one takes on models